In [1]:
import numpy as np
import sympy as sp
import math as mt
from sympy import Matrix, MatrixSymbol, Transpose, transpose, Function
from sympy import *
from sympy.physics.quantum.dagger import Dagger
from sympy.utilities.lambdify import lambdify, implemented_function
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm


#def symm(M):
 #   for i, j in zip(M, M.transpose()):
  #      print(i==j)

#Las masas en eV#
m_u = 2.16e+006
m_c = 1.27e+009
m_t = 172.69e+009

m_d = 4.67e+006
m_s = 93.4e+006
m_b = 4.18e+009

m_e = 0.510998995000e+006
m_mu = 105.6583755e+006
m_tau = 1776.86e+006

#Las deltas de masas cuadradas en eV^2#
Del21 = 7.53e-5
dDel21= 0.18e-5
Del32 = 2.437e-3
dDel32= 0.033e-3

V_ckm = Matrix([[0.97401, 0.22650, 0.00361],
         [0.22636, 0.97320, 0.04053],
         [0.00854, 0.03978, 0.999172]])

V_ckm.conjugate().transpose()

Matrix([
[0.97401, 0.22636,  0.00854],
[ 0.2265,  0.9732,  0.03978],
[0.00361, 0.04053, 0.999172]])

Aqui se define la expresión que se va a manipular:

$$M_D = \Omega \biggr[ X_1 \begin{pmatrix}
m_{u} & 0 & 0 \\
0 & m_{c} & 0 \\
0 & 0 & m_{t} 
\end{pmatrix}+ X_2 
V^\dagger_{ckm}
\begin{pmatrix}
m_{d} & 0 & 0 \\
0 & m_{s} & 0 \\
0 & 0 &  m_{b}
\end{pmatrix} V_{ckm} + X_3 \begin{pmatrix}
m_{e} & 0 & 0 \\
0 & m_{\mu} & 0 \\
0 & 0 & m_{\tau} 
\end{pmatrix}\biggr]$$

In [2]:
M_u= Matrix([[m_u, 0, 0],
      [0, m_c, 0],
      [0, 0, m_t]])

M_d= Matrix([[m_d, 0, 0],
      [0, m_s, 0],
      [0, 0, m_b]])

M_e= Matrix([[m_e, 0, 0],
      [0, m_mu, 0],
      [0, 0, m_tau]])

M_tex = V_ckm.conjugate().transpose() * M_d * V_ckm

Suponiendo que $\Omega=\frac{1}{1\times 10^{11}}$

In [3]:
Omega = 1e-11

X1 = Symbol('X1', real=True)
X2 = Symbol('X2', real=True)
X3 = Symbol('X3', real=True)

M_D = Omega*(X1*M_u+ X2*M_tex + X3*M_e)
M_D

Matrix([
[2.16e-5*X1 + 9.520970532707e-5*X2 + 5.10998995e-6*X3,                                0.0002302571632035*X2,                           0.00036540949493507*X2],
[                               0.0002302571632035*X2, 0.0127*X1 + 0.000953150594355*X2 + 0.001056583755*X3,                            0.0016983058887075*X2],
[                              0.00036540949493507*X2,                                0.0016983058887075*X2, 1.7269*X1 + 0.0417323427299709*X2 + 0.0177686*X3]])

In [4]:
M_D - M_D.conjugate().transpose()
# Como nota: Por alguna razón, al trasponer la matriz python identifica números reales que son exactamente iguales 
# como si fueran distintos por una pequeña cantidad (vease el siguiente bloque de codigo).
# Sin embargo hice de nuevo este proceso en mathematica y la matriz efectivamente es hermítica. 
# Lo unico a destacar esque por esta inconsistencia no puedo calcular en python los eigenvalores sin antes definir
# primero los valores numericos de X1 X2 X3. Habiendo hecho eso compare con los resultados en mathematica y se obtiene
# exactamente lo mismo. 

Matrix([
[0,                       0,                        0],
[0,                       0, -2.16840434497101e-19*X2],
[0, 2.16840434497101e-19*X2,                        0]])

In [5]:
print(M_D[5]-M_D.transpose().conjugate()[5])
print(M_D[7]-M_D.transpose().conjugate()[7])
print(M_D[5])
print(M_D.transpose().conjugate()[5])
print(M_D[7])
print(M_D.transpose().conjugate()[7])

-2.16840434497101e-19*X2
2.16840434497101e-19*X2
0.0016983058887075*X2
0.0016983058887075*X2
0.0016983058887075*X2
0.0016983058887075*X2


In [6]:
# En este bloque de codigo lo que hice fue acotar los valores de los eigenvalores de la matriz M_D basado en
# los valores númericos de lo que se hizo en la primera parte de la tesis de encontrar los rangos para m_3, m_2 y m_1
# Los valores que coloque junto a los i,j,k en la evaluación fueron puestos a prueba y error para acotar mas los calculos
# y que no fuera tan tardado el proceso. Añadi una barra de progreso para mi tranquilidad mental.

#for k in tqdm(range(524,660)):
 # for j in range(1,101):
  #  for i in range(73,96):
   #   eigenvalues = M_D.evalf(subs={X1:i*0.00001,X2:j*0.00001,X3:k*0.0001}).eigenvals()
    #  eig1 = list(eigenvalues.keys())[0]
     # eig2 = list(eigenvalues.keys())[1]
      #eig3 = list(eigenvalues.keys())[2]
      #if 2.55e-3 > eig3 > 2.47e-3:
       # if 7.9e-5 > eig2 > 6.8e-5: 
        #  if 3.9e-7 > eig1 > 3.2e-7:
         #   print('encontrao','X1:',i,'X2:',j,'X3:',k, eig3, eig2,eig1)
          #else:
           # print('no encontrao')


In [8]:
# Aqui solo estoy acotando lo de arriba pero duplico la celda para modificar libremente. Agregue la orden de exportar los datos a un txt.
with open("takis.txt", 'a+') as f:
  for k in tqdm(range(52,66)):
    for j in range(1,101):
      for i in range(73,96):
        eigenvalues = M_D.evalf(subs={X1:i*0.00001,X2:j*0.00001,X3:k*0.001}).eigenvals()
        eig1 = list(eigenvalues.keys())[0]
        eig2 = list(eigenvalues.keys())[1]
        eig3 = list(eigenvalues.keys())[2]
        if (0.05156 > eig3 > 0.05083):
          if (0.01417 > eig2 > 0.01346):
            if (0.01115 > eig1 > 0.01043):
              #print(f'{i}, {j}, {k}, {eig1}, {eig2}, {eig3}',end='\n', sep=',', file=f)
                print(i, j, k)

100%|███████████████████████████████████| 14/14 [01:56<00:00,  8.30s/it]


In [ ]:
for k in tqdm(range(1,101)):
    for j in range(1,101):
        for i in range(1,101):
            eigenvalues = M_D.evalf(subs={X1:i*0.00000000001,X2:j*0.00000000001,X3:k*0.000000000001}).eigenvals()
            eig1 = list(eigenvalues.keys())[0]
            eig2 = list(eigenvalues.keys())[1]
            eig3 = list(eigenvalues.keys())[2]
            if (Del32 + dDel32) >=(eig3**2-eig2**2)>= (Del32 - dDel32):
                if (Del21 + dDel21)>= (eig2**2 - eig1**2) >= (Del21 - dDel21):
                    print('encontrao','X1:',i,'X2:',j,'X3:',k, eig3, eig2,eig1)

  8%|██████▉                                                                               | 8/100 [05:01<57:50, 37.73s/it]

In [313]:
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#import numpy as np

#DataAll1D = np.loadtxt("takis.txt", delimiter=",")

#X, Y = np.meshgrid(DataAll1D[:,0], DataAll1D[:,1])
#Z = np.tile(DataAll1D[:,2], (len(DataAll1D[:,2]), 1))

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter3D(X, Y, Z, c=Z, cmap='ocean')

#plt.show()

In [ ]:
# Este bloque de codigo era para aproximar los valores pero no le di muchas vueltas. tal vez luego.

#guess1 = 0.00000
#guess2 = 0.00000
#increment = 0.000001
#epsilon = 0.00001


#while abs((list(eigenvalues.keys())[1])**2 - list(eigenvalues.keys())[0]**2 - dm_21) >= epsilon: 
 #   guess1+=increment 
  #  guess2+=increment
  
#if abs((list(eigenvalues.keys())[1])**2 - list(eigenvalues.keys())[0]**2 - dm_21) >= epsilon: 
 #   print("Failed on the cube root of",dm_21) 
#else: 
 #   print(guess1, guess2,"is close to",dm_21)  